In [ ]:
from anthropic import AnthropicBedrock
from compiler.core import Compiler
from langfuse import Langfuse
from fsm_core.helpers import solve_agent
from fsm_core import common, typespec, drizzle, typescript, handler_tests, handlers

import statemachine

In [3]:
langfuse_client = Langfuse()
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
m_claude = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


In [4]:
def not_impl(ctx):
    raise NotImplementedError("Not implemented")

def fail_state(ctx):
    print("FAILED")

In [5]:
class ActorContext:
    def __init__(self, compiler: Compiler):
        self.compiler = compiler


class TypespecActor:
    def __init__(self, m_claude: AnthropicBedrock, compiler: Compiler, langfuse_client: Langfuse):
        self.m_claude = m_claude
        self.compiler = compiler
        self.langfuse_client = langfuse_client

    def execute(self, application_description: str):
        start = typespec.Entry(application_description)
        result, _ = solve_agent(start, ActorContext(compiler), "solve_typespec", self.m_claude, self.langfuse_client)
        if result is None:
            raise ValueError("Failed to solve typespec")
        if not isinstance(result.data.inner, typespec.Success):
            raise Exception("Bad state: " + str(result.data.inner))
        return result.data.inner


class DrizzleActor:
    def __init__(self, m_claude: AnthropicBedrock, compiler: Compiler, langfuse_client: Langfuse):
        self.m_claude = m_claude
        self.compiler = compiler
        self.langfuse_client = langfuse_client

    def execute(self, typespec_definitions: str):
        start = drizzle.Entry(typespec_definitions)
        result, _ = solve_agent(start, ActorContext(compiler), "solve_drizzle", self.m_claude, self.langfuse_client)
        if result is None:
            raise ValueError("Failed to solve drizzle")
        if not isinstance(result.data.inner, drizzle.Success):
            raise Exception("Bad state: " + str(result.data.inner))
        return result.data.inner


class TypescriptActor:
    def __init__(self, m_claude: AnthropicBedrock, compiler: Compiler, langfuse_client: Langfuse):
        self.m_claude = m_claude
        self.compiler = compiler
        self.langfuse_client = langfuse_client

    def execute(self, typespec_definitions: str):
        start = typescript.Entry(typespec_definitions)
        result, _ = solve_agent(start, ActorContext(compiler), "solve_typescript", self.m_claude, self.langfuse_client)
        if result is None:
            raise ValueError("Failed to solve typescript")
        if not isinstance(result.data.inner, typescript.Success):
            raise Exception("Bad state: " + str(result.data.inner))
        return result.data.inner

In [6]:
typespec_actor = TypespecActor(m_claude, compiler, langfuse_client)
drizzle_actor = DrizzleActor(m_claude, compiler, langfuse_client)
typescript_actor = TypescriptActor(m_claude, compiler, langfuse_client)

In [7]:
machine: statemachine.State = {
    "on": {
        "START": "typespec",
    },
    "states": {
        "typespec": {
            "on": {
                "PROMPT": "adjust",
                "CONFIRM": "core",
            },
            "states": {
                "init": {
                    "invoke": {
                        "src": typespec_actor,
                        "input_fn": lambda ctx: ctx["application_description"],
                        "on_done": {
                            "target": "wait",
                            "actions": [lambda ctx, event: ctx.update({"typespec_schema": event})],
                        }
                    }
                },
                "adjust": {
                    "entry": [not_impl],
                },
                "wait": {},
            },
            "always": {
                "guard": lambda ctx: "typespec_schema" not in ctx,
                "target": "init",
            }
        },
        "core": {
            "states": {
                "drizzle": {
                    "invoke": {
                        "src": drizzle_actor,
                        "input_fn": lambda ctx: ctx["typespec_schema"].typespec,
                        "on_done": {
                            "target": "typescript",
                            "actions": [lambda ctx, event: ctx.update({"drizzle_schema": event})],
                        },
                        "on_error": {"target": "failure"},
                    }
                },
                "typescript": {
                    "invoke": {
                        "src": typescript_actor,
                        "input_fn": lambda ctx: ctx["typespec_schema"].typespec,
                        "on_done": {
                            "target": "complete",
                            "actions": [lambda ctx, event: ctx.update({"typescript_schema": event})],
                        },
                        "on_error": {"target": "failure"},
                    }
                },
                "handler_tests": {},
                "handlers": {},
            },
            "always": {
                "guard": lambda ctx: "typespec_schema" in ctx,
                "target": "drizzle",
            }
        },
        "complete": {
            "on": {
                "PROMPT": "edit_application",
            }
        },
        "failure": {
            "entry": [fail_state],
        },
        "edit_application": {
            "entry": [not_impl],
        },
    }
}

In [8]:
fsm = statemachine.StateMachine(machine, {"application_description": "Make me a greeting bot"})

In [9]:
fsm.send("START")

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


Processing transition: [] typespec
Processing transition: ['typespec'] init
Processing transition: ['typespec', 'init'] wait


In [12]:
fsm.send("CONFIRM")

Processing transition: ['typespec', 'wait'] core
Processing transition: ['core'] drizzle
Processing transition: ['core', 'drizzle'] typescript
Processing transition: ['core', 'typescript'] complete


In [14]:
fsm.context

{'application_description': 'Make me a greeting bot',
 'typespec_schema': <fsm_core.typespec.Success at 0xffffa1fef440>,
 'drizzle_schema': <fsm_core.drizzle.Success at 0xffffa22ad070>,
 'typescript_schema': <fsm_core.typescript.Success at 0xffffa0001af0>}

In [15]:
fsm.stack_path

['complete']